# K-means Clustering
## K-means = Unsupervised Learning zum Clustern von Datenpunkten


1. Spezifikation der Anzahl an Clustern k

1. Initialisierung der Zentroiden der Cluster

1. Bestimmung des nächsten Zentroiden für jeden Datenpunkt (Labeling)

1. Verschiebung der Centroiden in die Mitte der ihnen zugehörenden Datenpunktmenge

1. Wiederholung des Labelns und Verschiebens, bis keine Verschiebung mehr erfolgt

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_blobs
from IPython.display import clear_output

import warnings

warnings.filterwarnings("ignore")

K = 4
CLUSTER_STD = 2.5

In [ ]:
data = pd.DataFrame(
    make_blobs(
        n_samples=20000,
        n_features=2,
        centers=K,
        cluster_std=CLUSTER_STD,
        random_state=42,
    )[0],
    columns=["feature_1", "feature_2"],
)
labels_init = make_blobs(
    n_samples=20000, n_features=2, centers=K, cluster_std=CLUSTER_STD, random_state=42
)[1]
plt.figure(figsize=(10, 7))
plt.title("Sample data")
sns.set_theme(style="whitegrid")
sns.scatterplot(
    data=data,
    x="feature_1",
    y="feature_2",
    s=10,
    alpha=0.5,
    hue=labels_init,
    palette="viridis",
)
plt.show()

## Wie baue ich einen KMeans Algorithmus?

1. Daten skalieren

1. Zentroiden zufällig initialisieren

1. Datenpunkte labeln

1. Zentroiden updaten

1. 3.+4. wiederholden

## 1. Daten skalieren

In [ ]:
data = ((data - data.min()) / (data.max() - data.min())) * 9 + 1
data.head()

## 2. Zentroiden zufällig initialisieren

In [ ]:
def random_centroids(data, k):
    centroids = []
    for i in range(k):
        centroid = data.apply(lambda x: float(x.sample()))
        centroids.append(centroid)
    return pd.concat(centroids, axis=1)


centroids = random_centroids(data, k=K)
centroids

## 3. Datenpunkte labeln

Dies geschieht über die Berechnung des euklidschen Abstandes jedes Datenpunktes zu jedem Zentroiden. Jeder Datenpunkt erhält das Labes des Zentroiden, der sich am nächsten bei ihm befindet zugewiesen.

$
    {\displaystyle d(p,q)=\|q-p\|_{2}={\sqrt {(q_{1}-p_{1})^{2}+\cdots +(q_{n}-p_{n})^{2}}}={\sqrt {\sum _{i=1}^{n}(q_{i}-p_{i})^{2}}}}
$

```python
np.sqrt(((data - centroids.iloc[:, 0]) ** 2).sum(axis=1))
```

In [ ]:
def get_labels(data, centroids):
    # calculate datapoint distances to centroids (columns)
    distances = centroids.apply(lambda x: np.sqrt(((data - x) ** 2).sum(axis=1)))

    # find column min distance for each row
    return distances.idxmin(axis=1)

In [ ]:
labels = get_labels(data, centroids)
labels.value_counts()

In [ ]:
# visualize initial centroids and labels and data as subplots

fig, ax = plt.subplots(1, 2, figsize=(15, 7))
ax[0].set_title("labeled data with initial centroids")
sns.scatterplot(
    data=data,
    x="feature_1",
    y="feature_2",
    s=10,
    alpha=0.5,
    hue=labels,
    palette="viridis",
    ax=ax[0],
)
sns.scatterplot(
    data=centroids.T,
    x="feature_1",
    y="feature_2",
    s=50,
    alpha=0.8,
    color="black",
    ax=ax[0],
)
ax[1].set_title("Real labels")
sns.set_theme(style="whitegrid")
sns.scatterplot(
    data=data,
    x="feature_1",
    y="feature_2",
    s=10,
    alpha=0.5,
    hue=labels_init,
    palette="viridis",
    ax=ax[1],
)
plt.show()

## 4. Zentroiden updaten

### Berechnung des mittleren geometrischen Abstandes einer jeden Spalte in jeder erstellten Datenpunktmenge


$
    {\displaystyle \left(\prod _{i=1}^{n}a_{i}\right)^{\frac {1}{n}}={\sqrt[{n}]{a_{1}a_{2}\cdots a_{n}}}}
$


oder das logarithmisch skalierte geometrische Mittel


$
 {\displaystyle \exp {\left({{\frac {1}{n}}\sum \limits _{i=1}^{n}\ln a_{i}}\right)}}
$


python implementation:
```python
np.exp(np.log(x).mean())
```

In [ ]:
def new_centroids(data, labels):
    return data.groupby(labels).apply(lambda x: np.exp(np.log(x).mean())).T

In [ ]:
def plot_clusters_sample(data, centroids, labels, new_centroids):
    plt.figure(figsize=(10, 7))
    plt.title("Sample data")
    sns.set_theme(style="whitegrid")
    sns.scatterplot(
        data=data,
        x="feature_1",
        y="feature_2",
        s=10,
        alpha=0.5,
        hue=labels,
        palette="viridis",
    )

    # Increase the size of centroids
    sns.scatterplot(
        data=centroids.T, x="feature_1", y="feature_2", s=200, alpha=0.5, color="black"
    )

    # Draw arrow from old centroid to new centroid
    for i in range(centroids.columns.size):
        plt.arrow(
            centroids.iloc[0, i],
            centroids.iloc[1, i],
            new_centroids.iloc[0, i] - centroids.iloc[0, i],
            new_centroids.iloc[1, i] - centroids.iloc[1, i],
            head_width=0.2,
            head_length=0.2,
            fc="black",
            ec="black",
        )
    plt.show()


plot_clusters_sample(data, centroids, labels, new_centroids(data, labels))

## 5. Wiederholung des Labelns und Verschiebens


### Visualisierung der Iterationen

In [ ]:
def plot_clusters(data, labels, centroids, iteration, clear=True):
    if clear:
        clear_output(wait=True)

    plt.figure(figsize=(10, 7))

    plt.title(f"Iteration: {iteration}")

    sns.set_theme(style="whitegrid")
    sns.scatterplot(
        data=data,

        x="feature_1",

        y="feature_2",

        s=10,

        alpha=0.5,

        hue=labels,

        palette="viridis",
    )
    sns.scatterplot(

        data=centroids.T, x="feature_1", y="feature_2", s=100, alpha=0.5, color="black"
    )

    plt.show()

### Finaler KMeans Algorithmus mit Visualisierung

In [ ]:
max_iterations = 100

centroids = random_centroids(data, K)
old_centroids = pd.DataFrame()
iteration = 1

while iteration < max_iterations and not centroids.equals(old_centroids):
    old_centroids = centroids

    labels = get_labels(data, centroids)
    centroids = new_centroids(data, labels)
    plot_clusters(data, labels, centroids, iteration, clear=True)
    iteration += 1